<a href="https://colab.research.google.com/github/yahito/ml-skillbox/blob/master/10_10_2_train-ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# TensorFlow Object Detection: Обучение модели

https://github.com/tensorflow/models/tree/master/research/object_detection

### Монтирование Google Drive
Для данного демо нам понадобятся несколько ноутбуков, которые должны работать с одними и теме же данными. Поэтому, чтобы все ноутбуки имели доступ к нужным данным, нам будет необходимо подключить диск Google Drive и сохранять все данные на нём (включая данные, скачанные из интернета).

Для монтирования диска нужно выполнить данный блок, перейти по ссылке, получить код, скопировать его в поле ниже (в этом блоке) и нажать Enter

После монтирования диск будет находиться здесь: `/content/drive/My Drive`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Рабочая директория
Все данные будем хранить в директории `/content/drive/My Drive/tf_od_demo` (TensorFlow Object Detection Demo)

Директория должна быть уже создана (в предыдущем ноутбуке)

In [ ]:
%cd "/content/drive/My Drive/tf_od_demo"

/content/drive/My Drive/tf_od_demo


### Загрузка предобученной модели
Во время обучения мы будем делать fine-tuning предобученной модели. Для этого необходимо загрузить веса соответствующей модели `ssd_mobilenet_v1_coco`. Из названия следует, что детекционная архитектура -- `SSD`, базовая CNN модель -- `MobileNet_v1`, предобучена на датасете COCO.

Этот шаг нужно сделать один раз (не повторять, если модель уже была скачана ранее).

In [ ]:
if True:
    !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
    !tar -xzf ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz

--2023-04-19 18:52:10--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.4.128, 2404:6800:4003:c11::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.4.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20518283 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.57M  40.4MB/s    in 0.5s    

2023-04-19 18:52:11 (40.4 MB/s) - ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz’ saved [20518283/20518283]



### Подготовка данных для обучения
Ранее мы подготовили файл `train_data.record`, в котором содержится обучающий датасет.
Кроме него для обучения необходимо подготовить еще два файла (они также находятся в папке `my_data`)

* `label_map.pbtxt` (в нашем случае `cube_label_map.pbtxt`) - файл с описанием классов. В нашем случае у нас всего один класс.

* `pipeline.config` - файл, содержищий различне гиперпарамтеры и настройки обучения. Важные параметры: `num_classes`, `fine_tune_checkpoint`, `label_map_path`, `input_path`. В данном примере и для обучения и для теста используется один и тот же датасет (см. блоки параметров `train_input_reader`, `eval_input_reader`)

### Запуск обучения
После того, как мы подготовили все данные, запустить обучение очень просто: надо просто запустить скрипт `model_main.py` и передать ему соответствующие параметры.

*   Путь к файлу `pipeline.config`
*   Директорию, для записи выхода обучения (`model_dir`)
*   Количество итераций обучения (`num_train_steps`)



In [ ]:
!pip install lvis


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow_io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 42.9 MB/s eta 0:00:00


In [ ]:
!pip install tf-models-official

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 kB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 81.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=7fb6f5fd15da1792593485e65f452888d88ab97de56f33e7e2

In [ ]:
%%time

if False:
    # Delete output directory
    !rm -rf my_data/output

!export PYTHONPATH=$PYTHONPATH:models/research:models/research/slim ; python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=my_data/pipeline.config \
    --model_dir=my_data/output \
    --num_train_steps=10000 \
    --alsologtostderr

2023-04-19 19:32:39.249063: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 19:32:40.260214: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2023-04-19 19:32:43.153939: I tensorflow

### Заморозка модели
Для того, чтобы использовать обученную модель в режиме инференса, её нужно "заморозить". Другими словами - подготовить к инференсу. Для этого надо запустить скрипт `export_inference_graph.py` и передать соответствующие парамтеры.

* Путь к файлу `pipeline.config`
* Путь к весам обученной модели (`trained_checkpoint_prefix`)
* Путь к директории для сохранения замороженной модели (`output_directory`)

In [ ]:
!export PYTHONPATH=$PYTHONPATH:models/research:models/research/slim ; python models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=my_data/pipeline.config \
    --trained_checkpoint_prefix=my_data/output/model.ckpt-10000 \
    --output_directory=my_data/output/frozen/

python3: can't open file '/content/models/research/object_detection/export_inference_graph.py': [Errno 2] No such file or directory
